# 3분 딥러닝 Seq2Seq 튜토리얼
이 노트북은 [골빈해커의 3분 딥러닝 텐서플로맛](http://book.naver.com/bookdb/book_detail.nhn?bid=12556028)에서 제공하는 소스 코드를 공부한 것을 정리한 곳입니다.<br>
소스 출처는 [골빈해커님의 깃허브](https://github.com/golbin/TensorFlow-Tutorials)입니다.

In [1]:
# -*- coding: euc-kr -*-

In [2]:
# 챗봇, 번역, 이미지 캡셔닝등에 사용되는 시퀀스 학습/생성 모델인 Seq2Seq 을 구현해봅니다.
# 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어봅니다.
import tensorflow as tf
import numpy as np

# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd']
#       to   -> ['t', 'o', 'P', 'P']
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(char_arr)

In [3]:
print(char_arr)
print(len(char_arr))
print(num_dic)
print(len(num_dic))

['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xeb', '\x8b', '\xa8', '\xec', '\x96', '\xb4', '\xeb', '\x82', '\x98', '\xeb', '\xac', '\xb4', '\xeb', '\x86', '\x80', '\xec', '\x9d', '\xb4', '\xec', '\x86', '\x8c', '\xeb', '\x85', '\x80', '\xed', '\x82', '\xa4', '\xec', '\x8a', '\xa4', '\xec', '\x82', '\xac', '\xeb', '\x9e', '\x91']
65
{'\x80': 52, '\x82': 60, '\x85': 51, '\x86': 48, '\xed': 53, '\x8b': 30, '\x8a': 57, '\x8c': 49, '\xec': 59, '\x91': 64, '\x96': 33, '\x98': 37, '\x9d': 45, '\x9e': 63, '\xa4': 58, '\xa8': 31, '\xac': 61, '\xb4': 46, 'E': 1, 'P': 2, 'S': 0, 'a': 3, 'c': 5, 'b': 4, 'e': 7, 'd': 6, 'g': 9, 'f': 8, 'i': 11, 'h': 10, 'k': 13, 'j': 12, 'm': 15, 'l': 14, 'o': 17, 'n': 16, 'q': 19, 'p': 18, 's': 21, 'r': 20, 'u': 23, 't': 22, 'w': 25, 'v': 24, 'y': 27, 'x': 26, 'z': 28, '\xeb': 62}
48


디버깅 결과 한글 인코딩 문제로 인해 num_dic의 길이가 왜인지 축소되었다. 그래서 u''를 달아서 일단 길이를 맞췄는데 아래에서도 계속 오류가 발생하는 중이다.

In [4]:
# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        input_ = [num_dic[n] for n in seq[0]]
        #print('input: ',input_)
        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        #print('output: ',output)
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙인다.
        target = [num_dic[n] for n in (seq[1] + 'E')]
        #print('target: ',target)
        input_batch.append(np.eye(dic_len)[input_])
        #print(input_batch)
        output_batch.append(np.eye(dic_len)[output])
        #print(output_batch)
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)
        #print(target_batch)
    return input_batch, output_batch, target_batch


#########
# 옵션 설정
######
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
n_class = n_input = dic_len


#########
# 신경망 모델 구성
######
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
# [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])


# 인코더 셀을 구성한다.
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)

# 디코더 셀을 구성한다.
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)


model = tf.layers.dense(outputs, n_class, activation=None)


cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


#########
# 신경망 모델 학습
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')


#########
# 번역 테스트
######
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]
    print(result[0])
    print(decoded)

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated


print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))

('Epoch:', '0001', 'cost =', '4.236123')
('Epoch:', '0002', 'cost =', '3.600368')
('Epoch:', '0003', 'cost =', '2.712858')
('Epoch:', '0004', 'cost =', '1.870546')
('Epoch:', '0005', 'cost =', '1.452936')
('Epoch:', '0006', 'cost =', '1.223923')
('Epoch:', '0007', 'cost =', '0.754670')
('Epoch:', '0008', 'cost =', '0.607824')
('Epoch:', '0009', 'cost =', '0.569371')
('Epoch:', '0010', 'cost =', '0.515858')
('Epoch:', '0011', 'cost =', '0.363921')
('Epoch:', '0012', 'cost =', '0.513453')
('Epoch:', '0013', 'cost =', '0.261488')
('Epoch:', '0014', 'cost =', '0.331711')
('Epoch:', '0015', 'cost =', '0.197967')
('Epoch:', '0016', 'cost =', '0.246385')
('Epoch:', '0017', 'cost =', '0.159222')
('Epoch:', '0018', 'cost =', '0.168963')
('Epoch:', '0019', 'cost =', '0.128699')
('Epoch:', '0020', 'cost =', '0.132146')
('Epoch:', '0021', 'cost =', '0.098102')
('Epoch:', '0022', 'cost =', '0.061549')
('Epoch:', '0023', 'cost =', '0.084851')
('Epoch:', '0024', 'cost =', '0.108292')
('Epoch:', '0025

ValueError: 'E' is not in list

한글 관련 문제를 해결해야 한다.<br>
그런데 나는 seq2seq을 이해하기 위한 게 주된 목표이지,<br>
인코딩 문제를 해결하는 게 1차 목표가 아니라서 이 코드에 대한 개선은 저녁 시간대에 해야겠다.<br>
관련 링크 혹은 해결하면서 발견한 링크.
<br>[골빈해커 Seq2Seq 소스 코드](https://github.com/golbin/TensorFlow-Tutorials/blob/master/10%20-%20RNN/03%20-%20Seq2Seq.py)
<br>[구글 Colab](http://jybaek.tistory.com/686)
<br>[주피터 다중 커널 개념](http://blog.nacyot.com/articles/2015-05-08-jupyter-multiple-pythons/)
<br>[numpy.eye](https://docs.scipy.org/doc/numpy/reference/generated/numpy.eye.html)
<br>[파이썬 내장함수, enumerate](https://wikidocs.net/32)
<br>[파이썬 한글 인코딩](https://soooprmx.com/archives/4912)
<br>[파이썬 한글 인코딩2](http://drcarter.tistory.com/47)